In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vikas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vikas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\vikas\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
import json
import random

In [3]:
#from keras.models import load_model
from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5')

intents = json.loads(open('intents.json', encoding="utf8").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

C:\Users\vikas\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    print("sentence words ",sentence_words)
    return sentence_words

In [5]:
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)

    bag = [0]*len(words)

    for s in sentence_words:
        for i,w in enumerate(words):
            if(w==s):
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return(np.array(bag))

In [6]:
def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    print("p = ",p)
    res = model.predict(np.array([p]))[0]
    print("res = ",res)
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    print("result = ",results)
    results.sort(key=lambda x: x[1], reverse=True)
    result_list = []
    for r in results:
        result_list.append({"intent":classes[r[0]], "probability": str(r[1])})

    return result_list

In [7]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    print("tag = ",tag)
    list_of_intents = intents_json['intents']
    print("list of intents = ",list_of_intents)
    for i in list_of_intents:
        if(i['tag']==tag):
            result = random.choice(i['responses'])
            break

    return result

In [8]:
def chatbot_response(msg):
    ints = predict_class(msg,model)
    print("ints = ",ints)
    try:
        res = getResponse(ints, intents)

    except:
        print("An Exception occurred!")
        res = "I can't answer this query. I am really sorry but I have no information about it. Please provide this in feedback for my improvement"
    
    print("chatbot response =",res)
    return res

**Flask Code**

In [ ]:
import logging
from flask import Flask, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.route("/", methods=['GET'])
def hello():
    return jsonify({"health": "server is running successfully"})

def decrypt(msg):
    string = msg
    new_string = string.replace("+", " ")
    return new_string

@app.route("/query/<sentence>")
def query_chatbot(sentence):
    # Decrypt message
    dec_msg = decrypt(sentence)
    
    response = chatbot_response(dec_msg)

    json_obj = jsonify({"top": {"res": response}})
    
    return json_obj
    
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


sentence words  ['hi']
p =  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step


INFO:werkzeug:127.0.0.1 - - [08/May/2024 13:04:09] "GET /query/hi HTTP/1.1" 200 -


res =  [2.25863550e-05 4.80765570e-03 6.07787285e-07 7.89626938e-07
 5.95115416e-05 6.90726210e-06 8.21075446e-06 3.64079483e-06
 1.88174454e-04 2.02021492e-05 3.58828800e-07 5.42504438e-07
 1.10460576e-02 2.18044018e-08 3.89002380e-05 2.36687143e-07
 2.35740208e-06 7.84136830e-07 9.81029661e-05 9.61819969e-05
 3.40800412e-04 8.44277750e-08 1.12581938e-06 1.86144625e-05
 1.39622318e-06 1.54934139e-06 6.91111345e-05 9.71247792e-01
 2.46885952e-06 1.92624839e-07 8.00220192e-08 3.04788282e-05
 9.66507505e-05 5.78438921e-05 6.72617243e-05 6.23774232e-09
 3.21231454e-07 1.00310615e-06 4.43126692e-06 1.79443559e-05
 2.45895761e-04 5.94629534e-03 7.64004380e-06 1.50402741e-07
 5.91345497e-06 5.64829236e-07 4.71875137e-06 4.86237695e-03
 3.70050875e-05 2.53831735e-04 6.15314411e-10 1.49455772e-07
 5.31228430e-08 2.52267478e-06 7.79124987e-10 6.49234971e-06
 9.61656315e-06 1.12802695e-06 1.69416289e-05 2.00518821e-06
 1.33264621e-04 4.26591384e-07 2.28631958e-08 1.35858454e-06
 1.00610974e-04]


...

res =  [0.0000000e+00 5.2019918e-35 0.0000000e+00 1.1300874e-31 0.0000000e+00
 9.2323429e-24 0.0000000e+00 0.0000000e+00 2.2195069e-34 7.8083823e-32
 0.0000000e+00 6.7932160e-37 1.6292651e-37 2.0848160e-37 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 5.6660242e-34 0.0000000e+00
 7.4720507e-30 7.1806524e-37 1.2837195e-34 1.1460074e-30 1.0920876e-26
 9.3579488e-36 0.0000000e+00 0.0000000e+00 0.0000000e+00 9.7746368e-34
 0.0000000e+00 8.3827891e-34 1.3879682e-34 0.0000000e+00 9.7862026e-34
 0.0000000e+00 4.2464690e-36 0.0000000e+00 3.6690175e-36 9.8181685e-35
 3.5968208e-34 3.0221921e-33 0.0000000e+00 8.5505106e-33 1.5859233e-37
 1.0000000e+00 0.0000000e+00 0.0000000e+00 3.6506329e-35 1.0465930e-30
 0.0000000e+00 0.0000000e+00 8.6075103e-35 0.0000000e+00 1.0235709e-36
 2.0963477e-37 0.0000000e+00 1.3872647e-33 7.6035979e-38 2.1441513e-25
 2.9459381e-34 0.0000000e+00 0.0000000e+00 4.5621410e-30 0.0000000e+00]
result =  [[45, 1.0]]
ints =  [{'intent': 'principal', 'probability':

INFO:werkzeug:127.0.0.1 - - [08/May/2024 14:22:29] "GET /query/faculty+information HTTP/1.1" 200 -


res =  [1.66958339e-21 8.39600454e-27 2.30324176e-20 3.79922924e-20
 2.18274001e-18 5.68475641e-17 1.17603475e-21 1.67077016e-19
 2.51068073e-25 3.17378905e-23 2.51924959e-24 1.33332837e-14
 3.23560569e-25 1.64538098e-20 3.18132646e-13 2.07748127e-21
 2.34656587e-19 4.80943529e-15 3.40411197e-21 1.00000000e+00
 5.85432492e-24 6.61583754e-28 2.75764202e-26 2.73577877e-23
 6.22136239e-20 1.56726641e-23 1.10804123e-20 2.99875438e-17
 9.91028622e-27 6.69039087e-24 1.07716501e-24 2.56408308e-15
 2.24462200e-29 4.52126714e-24 3.17124219e-19 2.76776089e-25
 1.56022714e-16 1.31184556e-15 1.49594962e-27 1.45580505e-26
 1.24411127e-16 9.92058487e-17 9.28641346e-18 6.13499690e-25
 2.04842140e-27 1.64190329e-25 2.71190929e-25 1.92352140e-19
 3.24727720e-18 1.17713639e-25 1.35524565e-36 7.81177338e-26
 8.08020204e-22 1.68703159e-26 9.36758618e-22 1.38339102e-18
 4.47752889e-26 4.84808457e-21 7.60206226e-15 9.40493148e-22
 1.89214657e-20 8.35708586e-19 4.24189075e-27 2.01003829e-24
 2.20075641e-19]


INFO:werkzeug:127.0.0.1 - - [08/May/2024 14:22:46] "GET /query/university+name HTTP/1.1" 200 -


res =  [4.33360847e-10 1.20609717e-10 3.25309168e-10 1.77313746e-06
 6.41846270e-07 4.36142670e-07 4.66042593e-06 7.70740622e-13
 7.58842919e-11 3.35960149e-11 2.82159903e-07 1.66986762e-08
 1.00431448e-10 7.91437955e-08 3.12208370e-10 4.44490800e-09
 7.34904715e-09 1.53986548e-07 3.18663235e-10 5.61201450e-05
 4.28841546e-11 1.64324266e-11 4.84201524e-12 5.42334797e-12
 2.06638157e-04 6.47357862e-11 4.11241734e-03 6.77150946e-10
 5.91890448e-09 7.08745063e-08 1.36583819e-13 7.18645481e-08
 1.50602295e-13 1.64157837e-12 3.27072570e-07 3.43307696e-12
 2.08186263e-11 2.05179418e-07 3.72749609e-11 1.17150853e-12
 1.51821581e-07 2.06695440e-05 1.73630816e-08 1.05580715e-04
 1.55475792e-08 3.02840917e-08 3.43476185e-11 2.19631019e-12
 6.10430561e-07 7.29150873e-09 5.43398639e-14 9.92326665e-10
 7.89724008e-10 9.78981913e-12 2.14093743e-11 5.73790487e-07
 2.39879866e-10 7.19063019e-06 4.20544978e-07 2.66699840e-06
 1.71470198e-08 9.95478094e-01 3.18319156e-11 1.34833056e-09
 1.14377960e-11]
